In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}

url = "https://www.melon.com/chart/index.htm"
res = requests.get(url, headers=headers)
res

<Response [200]>

In [3]:
html = res.text
len(html)

457861

In [4]:
soup = BeautifulSoup(html, "html.parser")

song_list = soup.select("span.rank")
len(song_list)

101

In [5]:
# 한 곡에 대한 test
tr_list = soup.select('tbody>tr')
test_tr = tr_list[0]

In [6]:
# 곡 일련번호
song_num = test_tr['data-song-no']
# 곡 순위
rank = test_tr.select_one('span.rank').string
# 앨범명
album_name = test_tr.select_one('.service_list_song .ellipsis.rank03 a').string
# 곡명
song_name = test_tr.select_one('.service_list_song .ellipsis.rank01 a').string
# 가수
artist = test_tr.select_one('.service_list_song .ellipsis.rank02 a').string
# 커버이미지 주소
coverimg_url = test_tr.select_one('a img')['src']
# 좋아요
# "contsLike">[0]>"SUMMCNT"
like_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_num}'
like_html = requests.get(like_url, headers=headers).text
likes = int(json.loads(like_html)['contsLike'][0]['SUMMCNT'])

song_num, rank, album_name, song_name, artist, coverimg_url, likes

('36599950',
 '1',
 "NewJeans 2nd EP 'Get Up'",
 'Super Shy',
 'NewJeans',
 'https://cdnimg.melon.co.kr/cm2/album/images/112/81/456/11281456_20230706180841_500.jpg/melon/resize/120/quality/80/optimize',
 102204)

In [7]:
# 모든 곡에 적용하기 위해 함수로 만들어서 실행
def get_melon_info(tr):
  # 곡 일련번호
  song_num = tr['data-song-no']
  # 곡 순위
  rank = tr.select_one('span.rank').string
  # 앨범명
  album_name = tr.select_one('.service_list_song .ellipsis.rank03 a').string
  # 곡명
  song_name = tr.select_one('.service_list_song .ellipsis.rank01 a').string
  # 가수
  artist = tr.select_one('.service_list_song .ellipsis.rank02 a').string
  # 커버이미지 주소
  coverImg_url = tr.select_one('a img')['src']
  # 좋아요
  # "contsLike">[0]>"SUMMCNT"
  like_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_num}'
  like_html = requests.get(like_url, headers=headers).text
  likes = int(json.loads(like_html)['contsLike'][0]['SUMMCNT'])

  song_info = {
      '곡일련번호': [song_num],
      '순위' : [rank],
      '앨범' : [album_name],
      '곡명' : [song_name],
      '가수' : [artist],
      '커버이미지_주소' : [coverImg_url],
      '좋아요' : [likes]
  }
  song_info_df = pd.DataFrame(song_info)

  return song_info_df

In [12]:
def 멜론차트_가져오기():
  # 함수 실행
  tr_list = soup.select('tbody>tr')
  melon_chart = pd.DataFrame(columns = {
      '곡일련번호' : [],
      '순위' : [],
      '앨범' : [],
      '곡명': [],
      '가수' : [],
      '커버이미지_주소' : [],
      '좋아요' : []
      })

  for tr in tr_list:
    melon_chart = pd.concat([melon_chart, get_melon_info(tr)], ignore_index = True)

  return melon_chart

In [ ]:
df = 멜론차트_가져오기()
print(df.shape)
df.head()